## Setup Environment

In [ ]:
!pip install ultralytics
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.4/881.4 kB 6.0 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO , solutions
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import os

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
!pip install pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 779.6 kB/s eta 0:00:00


In [ ]:
import pymysql

# Database connection details
RDS_HOST = "database-2.cnqamusmkwon.eu-north-1.rds.amazonaws.com"
RDS_PORT = 3306  # MySQL default port
DB_USER = "root"
DB_PASSWORD = "mkmk162345"
DB_NAME = "traffic"


# Function to connect to the database
def get_connection():
    return pymysql.connect(
        host=RDS_HOST,
        port=RDS_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        cursorclass=pymysql.cursors.DictCursor
    )

# Function to increment road_in, calculate road_current, and update the table
def increment_road_in(id_value, increment_value):
    try:
        connection = get_connection()
        with connection.cursor() as cursor:
            # Step 1: Retrieve the current data
            select_sql = "SELECT id, road_in, road_out, road_current FROM traffic_counter_road WHERE id = %s"
            cursor.execute(select_sql, (id_value,))
            result = cursor.fetchone()

        if result:
            # Step 2: Modify road_in and calculate road_current
            new_road_in = result['road_in'] + increment_value
            new_road_current = new_road_in - result['road_out']

            # Step 3: Update the table
            with connection.cursor() as cursor:
                update_sql = """
                UPDATE traffic_counter_road
                SET road_in = %s, road_current = %s
                WHERE id = %s
                """
                cursor.execute(update_sql, (new_road_in, new_road_current, id_value))

            # Commit the changes
            connection.commit()
            print(f"Updated road_in to {new_road_in} and road_current to {new_road_current} for id {id_value}")

    except pymysql.MySQLError as e:
        print(f"Error: {e}")

    finally:
        if connection:
            connection.close()

# Function to increment road_out, calculate road_current, and update the table
def increment_road_out(id_value, increment_value):
    try:
        connection = get_connection()
        with connection.cursor() as cursor:
            # Step 1: Retrieve the current data
            select_sql = "SELECT id, road_in, road_out, road_current FROM traffic_counter_road WHERE id = %s"
            cursor.execute(select_sql, (id_value,))
            result = cursor.fetchone()

        if result:
            # Step 2: Modify road_out and calculate road_current
            new_road_out = result['road_out'] + increment_value
            new_road_current = result['road_in'] - new_road_out

            # Step 3: Update the table
            with connection.cursor() as cursor:
                update_sql = """
                UPDATE traffic_counter_road
                SET road_out = %s, road_current = %s
                WHERE id = %s
                """
                cursor.execute(update_sql, (new_road_out, new_road_current, id_value))

            # Commit the changes
            connection.commit()
            print(f"Updated road_out to {new_road_out} and road_current to {new_road_current} for id {id_value}")

    except pymysql.MySQLError as e:
        print(f"Error: {e}")

    finally:
        if connection:
            connection.close()

# # Example usage:
# increment_road_in(1, 1)
# increment_road_out(1, 1)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load the YOLO Model

In [ ]:
model = YOLO('/content/YOLO_Model_v5.pt')

WARNING ⚠️ /content/YOLO_Model_v5.pt appears to require 'dill', which is not in Ultralytics requirements.
AutoInstall will run now for 'dill' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official Ultralytics model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['dill'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 3.6 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 3.0s, installed 1 package: ['dill']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



## Prepare the Video Capture

In [ ]:
# Prepare the video capture
video_path = '/content/drive/MyDrive/Dataset/IMG_9466.MOV'
cap = cv2.VideoCapture(video_path)

## Get Video Information

In [ ]:
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))

In [ ]:
print("Height:", height)
print("Width:", width)
print("FPS:", fps)

Height: 1584
Width: 2816
FPS: 59


In [ ]:
output_path = 'output_video.avi'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

In [ ]:
License_plate = set()

In [ ]:
box = (1650, 900, 2816, 1500)

In [ ]:
counter_Out = 0

In [ ]:
print(model.names)

{0: 'License_Plate', 1: 'cars', 2: 'motorcycle', 3: 'truck'}


In [ ]:
class_names = ['License Plate', 'Car', 'Motorcycle', 'Truck']
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO model on the frame
    results = model.track(frame, persist=True)

    for result in results:
        for boxes in result.boxes:

            # Get Object details
            bbox = boxes.xyxy[0].cpu().numpy()          # Bounding Box
            class_id = int(boxes.cls[0].cpu().numpy())  # Class ID
            conf = boxes.conf[0].cpu().numpy()          # Confidence

            # Check if boxes.id is not None before accessing it
            if boxes.id is not None:
                id = int(boxes.id[0].cpu().numpy())     # Object ID
            else:
                # Handle the case where id is None
                id = -1

            x1, y1, x2, y2 = map(int, bbox)
            cropped_object = frame[y1:y2, x1:x2]

            # Draw BBox
            cv2.rectangle(frame, (x1, y1), (x2, y2), (208, 38, 7), 3)

            label = f'ID: {id}, class: {class_names[class_id]} Conf: {conf:.2f}'
            # Add labels for each car
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (208, 38, 7), 2, cv2.LINE_AA)
            if x1 >= box[0] and y1 >= box[1] and x2 <= box[2] and y2 <= box[3]:
                if id not in License_plate:
                    License_plate.add(id)
                    increment_road_out(1, 1)
                    save_path = os.path.join('/content/Images', f"{counter_Out}.jpg")  # Save with object name and index
                    counter_Out += 1
                    # Save the cropped image
                    cv2.imwrite(save_path, cropped_object)

    # cv2.rectangle(frame, box[:2], box[2:], (0, 0, 255), 2)


    video_writer.write(frame)

    # Display each frame
    # cv2_imshow(frame)


Streaming output truncated to the last 5000 lines.

0: 384x640 1 License_Plate, 1 cars, 561.1ms
Speed: 4.0ms preprocess, 561.1ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 1 cars, 495.2ms
Speed: 4.9ms preprocess, 495.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 1 cars, 372.5ms
Speed: 3.7ms preprocess, 372.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 1 cars, 366.5ms
Speed: 3.7ms preprocess, 366.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 1 cars, 381.6ms
Speed: 4.4ms preprocess, 381.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 1 cars, 371.7ms
Speed: 6.4ms preprocess, 371.7ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 License_Plate, 1 cars, 386.9ms
Speed: 3.9ms preprocess, 386.9ms inference, 0.

In [ ]:
cap.release()
video_writer.release()
cv2.destroyAllWindows()
